# Sector Coupling in PyPSA

This tutorial covers how to model sector coupling in PyPSA, integrating different energy sectors (power, heat, transport demand, hydrogen) and their interactions. We'll explore how different sectors can be connected and optimized together.

## Sector Coupling Overview

Sector coupling involves connecting different energy sectors:

1. **Power Sector**: Electricity generation and distribution
2. **Heat Sector**: Industrial or district heating demand
3. **Transport Sector Demand**: Represented by loads on electricity (EVs) and hydrogen (FCEVs/Industry)
4. **Hydrogen Sector**: Production and storage

Let's create a network that demonstrates sector coupling:

In [ ]:
import pypsa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Create network with time series (24 hours)
network = pypsa.Network()
snapshots = pd.date_range("2024-01-01", periods=24, freq="h")
network.set_snapshots(snapshots)

# Add buses for different sectors
network.add("Bus", "Electricity Bus") # Default carrier is AC
network.add("Bus", "Heat Bus", carrier="heat")
# network.add("Bus", "Transport Bus", carrier="transport") # Removed - demand represented elsewhere
network.add("Bus", "Hydrogen Bus", carrier="hydrogen")

# Add carriers
network.add("Carrier", "Gas", co2_emissions=0.2) # tCO2 / MWh_th
network.add("Carrier", "Solar")
# Heat and Hydrogen carriers are implicitly created by their buses

## Adding Sector-Specific Components (Supply & Demand)

Let's add supply and demand components for each sector. We use fixed capacities and time-varying load profiles to focus on operational interactions.
We also add a backup gas generator for feasibility.

In [ ]:
# --- Power Sector Components ---
# Solar generator
network.add(
    "Generator",
    "Solar PV",
    bus="Electricity Bus",
    p_nom=100,  # Fixed capacity (MW)
    p_nom_extendable=False,
    marginal_cost=0, # Near-zero cost
    p_max_pu=pd.Series(
        [0.0, 0.0, 0.0, 0.0, 0.1, 0.3, 0.5, 0.7, 0.8, 0.9, 0.9, 0.8,
         0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
        index=network.snapshots
    ),
    carrier="Solar"
)

# Backup gas generator
network.add(
    "Generator",
    "Gas Backup",
    bus="Electricity Bus",
    p_nom=150,  # Fixed capacity, ensure it can cover peak load
    p_nom_extendable=False,
    marginal_cost=55,  # EUR/MWh_el
    carrier="Gas",
    efficiency=0.38
)

# Electricity Load (Residential/Commercial)
elec_load_profile = pd.Series([0.6, 0.5, 0.5, 0.5, 0.6, 0.7, 0.8, 0.8, 0.8, 0.7, 0.7, 0.7,
                               0.7, 0.7, 0.8, 0.9, 1.0, 1.0, 0.9, 0.8, 0.7, 0.6, 0.6, 0.6], index=network.snapshots) * 60  # Peak 60 MW
network.add("Load", "Electricity Demand", bus="Electricity Bus", p_set=elec_load_profile)

# EV Charging Load (Flexible Demand on Electricity Bus)
ev_load_profile = pd.Series([0.1, 0.1, 0.1, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.5, 0.4, 0.3,
                             0.3, 0.3, 0.4, 0.5, 0.7, 0.9, 1.0, 0.8, 0.5, 0.3, 0.2, 0.1], index=network.snapshots) * 25  # Peak 25 MW
network.add(
    "Load",
    "EV Charging Demand",
    bus="Electricity Bus",
    p_set=ev_load_profile,
    p_max_pu=1.2,  # Can charge up to 20% faster if needed/cheap
    p_min_pu=0.0   # Can stop charging
)

# --- Heat Sector Components ---
# (Removed Gas boiler - relying on Heat Pump / Storage)

# Heat Load (Flexible Demand)
heat_load_profile = pd.Series([0.8, 0.7, 0.6, 0.6, 0.7, 0.9, 1.0, 0.9, 0.8, 0.7, 0.6, 0.6,
                               0.6, 0.6, 0.7, 0.8, 0.9, 1.0, 1.0, 0.9, 0.8, 0.8, 0.8, 0.8], index=network.snapshots) * 40  # Peak 40 MWth
network.add(
    "Load",
    "Heat Demand",
    bus="Heat Bus",
    p_set=heat_load_profile,
    p_max_pu=1.1,
    p_min_pu=0.9
)

# --- Hydrogen Sector Components ---
# Hydrogen Load (e.g., Industry/Transport - Flexible)
h2_load_profile = pd.Series([0.8] * 8 + [1.0] * 8 + [0.7] * 8, index=network.snapshots) * 30 # Peak 30 MW_H2
network.add(
    "Load",
    "Hydrogen Demand",
    bus="Hydrogen Bus",
    p_set=h2_load_profile,
    p_max_pu=1.2,
    p_min_pu=0.7
)

## Adding Sector Coupling Technologies

Now let's add technologies that connect different sectors, using fixed capacities and operational costs.

In [ ]:
# Power-to-Heat: Heat pump
network.add(
    "Link",
    "Heat Pump",
    bus0="Electricity Bus",
    bus1="Heat Bus",
    p_nom=30,  # Fixed Capacity (MW_el input)
    p_nom_extendable=False,
    efficiency=3.0,  # Coefficient of Performance (COP)
    marginal_cost=1  # Small operational cost per MWh_el input
)

# Power-to-Hydrogen: Electrolyzer
network.add(
    "Link",
    "Electrolyzer",
    bus0="Electricity Bus",
    bus1="Hydrogen Bus",
    p_nom=50,  # Fixed Capacity (MW_el input)
    p_nom_extendable=False,
    efficiency=0.7,  # MWh_H2 / MWh_el
    marginal_cost=2  # Operational cost per MWh_el input
)

# Hydrogen-to-Power: Fuel Cell
network.add(
    "Link",
    "Fuel Cell",
    bus0="Hydrogen Bus",
    bus1="Electricity Bus",
    p_nom=40,  # Fixed Capacity (MW_H2 input)
    p_nom_extendable=False,
    efficiency=0.55,  # MWh_el / MWh_H2
    marginal_cost=3  # Operational cost per MWh_H2 input
)

## Adding Storage Options

Let's add storage options for different sectors.

In [ ]:
# Battery storage for power sector
network.add(
    "Store",
    "Battery Storage",
    bus="Electricity Bus",
    e_nom=100,  # Fixed Capacity (MWh)
    p_nom=20,  # Fixed Capacity (MW)
    e_nom_extendable=False,
    p_nom_extendable=False,
    e_cyclic=True,
    e_min_pu=0.1, e_max_pu=0.9,
    standing_loss=0.001,  # 0.1% per hour
    efficiency_store=0.95, efficiency_dispatch=0.95,
    marginal_cost=0.1  # Cost per MWh cycled
)

# Heat storage
network.add(
    "Store",
    "Heat Storage Tank",
    bus="Heat Bus",
    e_nom=200,
    p_nom=40,
    e_nom_extendable=False,
    p_nom_extendable=False,
    e_cyclic=True,
    e_min_pu=0.1, e_max_pu=0.9,
    standing_loss=0.01,  # 1% per hour
    efficiency_store=0.98, efficiency_dispatch=0.98,
    marginal_cost=0.01  # Low cost per MWh_th cycled
)

# Hydrogen storage
network.add(
    "Store",
    "Hydrogen Storage Tank",
    bus="Hydrogen Bus",
    e_nom=500,  # MWh_H2
    p_nom=50,  # MW_H2
    e_nom_extendable=False,
    p_nom_extendable=False,
    e_cyclic=True,
    e_min_pu=0.1, e_max_pu=0.9,
    standing_loss=0.001,  # 0.1% per hour
    efficiency_store=0.99, efficiency_dispatch=0.99,
    marginal_cost=0.05  # Cost per MWh_H2 cycled
)

## Adding Global Constraints (Optional)

Global constraints like CO2 limits can be added. PyPSA calculates CO2 based on carrier `co2_emissions` and generator `efficiency`. 
We will skip these for simplicity in this operational tutorial.
```python
# Example CO2 Limit 
# network.add("GlobalConstraint", "co2_limit", sense="<=", constant=5000) # tCO2 for the period
```
*Refer to PyPSA documentation for details.*

In [27]:
# Global constraints skipped for this example.
pass

## Optimizing the Coupled System

Let's solve the optimization problem (operational dispatch) and analyze the results.

In [ ]:
# Solve the optimization problem
status = None
condition = None
print("Attempting to solve optimization...")
try:
    status, condition = network.optimize()
    print(f"Solver Status: {status}, Termination Condition: {condition}")
except Exception as e:
    print(f"An error occurred during optimization: {e}")

# Analyze results only if optimization was successful
if status == "ok" and condition == "optimal":
    print(f"Optimization successful. Total operational cost: {network.objective:.2f}")

    try:
        # Plot generator dispatch by carrier
        plt.figure(figsize=(12, 6))
        p_by_carrier = network.generators_t.p.groupby(network.generators.carrier, axis=1).sum()
        # Ensure all expected carriers are present, add if missing with 0
        for carrier in network.carriers.index:
            if carrier not in p_by_carrier.columns and carrier in network.generators.carrier.unique():
                 p_by_carrier[carrier] = 0
        # Re-order columns for consistent plotting
        p_by_carrier = p_by_carrier.reindex(columns=sorted(p_by_carrier.columns))
        p_by_carrier.plot(kind="area", alpha=0.7)
        plt.title('Generator Dispatch by Carrier')
        plt.xlabel('Time')
        plt.ylabel('Power (MW)')
        plt.legend(title='Carrier')
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # Plot storage operation (energy levels)
        plt.figure(figsize=(12, 6))
        network.stores_t.e.plot()
        plt.title('Storage Energy Levels')
        plt.xlabel('Time')
        plt.ylabel('Energy (MWh or MWh_th or MWh_H2)')
        plt.legend(title='Storage')
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # Plot flows through coupling links (p0 = input power)
        plt.figure(figsize=(12, 6))
        network.links_t.p0.plot()
        plt.title('Power Input to Coupling Links')
        plt.xlabel('Time')
        plt.ylabel('Power (MW)')
        plt.legend(title='Link')
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # Plot flexible load dispatch vs setpoint
        plt.figure(figsize=(12, 6))
        # Identify flexible loads (those with p_max_pu != 1 or p_min_pu != 1)
        flexible_loads = network.loads[(network.loads.p_max_pu != 1) | (network.loads.p_min_pu != 1)].index
        if not flexible_loads.empty:
            for load in flexible_loads:
                network.loads_t.p[load].plot(style='-', label=f'{load} Dispatch')
                if isinstance(network.loads.p_set[load], pd.Series):
                    network.loads.p_set[load].plot(style=':', label=f'{load} Setpoint')
                else:
                    pd.Series(network.loads.p_set[load], index=network.snapshots).plot(style=':', label=f'{load} Setpoint')
            plt.title('Flexible Load Dispatch vs. Setpoint Profile')
            plt.xlabel('Time')
            plt.ylabel('Power (MW or MWth or MW_H2)')
            plt.legend()
            plt.grid(True)
            plt.tight_layout()
            plt.show()
        else:
            print("No flexible loads identified to plot.")

    except Exception as e:
        print(f"An error occurred during plotting: {e}")

elif status is not None:
    print("Skipping results analysis due to non-optimal solver status.")
else:
    print("Skipping results analysis because optimization did not run or failed.")

## Key Sector Coupling Concepts

1. **Sector Integration**
   - Utilizing diverse energy resources (renewables, gas) across sectors.
   - Connecting electricity, heat, hydrogen, and transport demands.
   - Technologies like heat pumps and electrolyzers link these sectors.

2. **Coupling Technologies**
   - Power-to-Heat (Heat Pumps for industrial/district heat)
   - Power-to-Hydrogen (Electrolysis for green hydrogen)
   - Hydrogen-to-Power (Fuel Cells for backup/grid services)
   - Power-to-Transport (EV Charging)

3. **Storage Options**
   - Batteries for electricity balancing.
   - Thermal Storage for heat demand management.
   - Hydrogen Storage for buffering hydrogen production/use.

4. **Flexibility**
   - Dispatchable generation (Gas Backup).
   - Flexible loads (EV charging, industrial processes).
   - Storage operation across different energy carriers.
   - Cross-sector optimization to minimize costs (e.g., using cheap electricity for hydrogen/heat).

## Benefits of Sector Coupling

1. **Enhanced Renewable Integration**: Better utilize variable renewables (like Solar) by storing energy or converting it to hydrogen/heat.
2. **Grid Stability**: Batteries and flexible demand help manage fluctuations.
3. **Decarbonization**: Reduce reliance on fossil fuels in heating and potentially transport via green hydrogen or direct electrification.
4. **Resource Optimization**: Use electricity efficiently across sectors based on cost and availability.

## Next Steps

This completes the PyPSA tutorial series. You now have a comprehensive understanding of how to use PyPSA for power system analysis, from basic network creation to storage, investment planning, and sector coupling.